<a href="https://colab.research.google.com/github/cpython-projects/da_2603/blob/main/lesson_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Легенда

Вы — аналитик данных в розничной компании, развивающей мобильное приложение. Команда продукта хочет понять:

* как пользователи взаимодействуют с приложением: сколько установок, сколько просматривают товары, сколько покупают;
* где теряются пользователи (на каком этапе воронки);
* какова эффективность разных каналов привлечения.

### Таблицы БД

**1. `app_sessions`** — установки приложения

| Поле                  | Тип SQL        | Описание                                        |
| --------------------- | -------------- | ----------------------------------------------- |
| `session_id`          | `VARCHAR(20)`  | Уникальный идентификатор установки              |
| `device_code`         | `VARCHAR(20)`  | Идентификатор устройства                        |
| `first_seen`          | `DATE`         | Дата установки                                  |
| `os_type`             | `VARCHAR(10)`  | Платформа (`iOS`, `Android`)                    |
| `acquisition_channel` | `VARCHAR(50)`  | Канал установки (`Organic`, `Facebook`, и т.д.) |
| `cpi_uah`             | `NUMERIC(6,2)` | Стоимость установки (Cost Per Install) в грн    |


Вот обновлённый блок лекции с пояснением про стоимость установки (CPI):

---

### 1. `app_sessions` — установки приложения

| Поле                  | Тип SQL        | Описание                                            |
| --------------------- | -------------- | --------------------------------------------------- |
| `session_id`          | `VARCHAR(20)`  | Уникальный идентификатор установки                  |
| `device_code`         | `VARCHAR(20)`  | Идентификатор устройства                            |
| `first_seen`          | `DATE`         | Дата установки                                      |
| `os_type`             | `VARCHAR(10)`  | Платформа (`iOS`, `Android`)                        |
| `acquisition_channel` | `VARCHAR(50)`  | Канал установки (`Organic`, `Facebook`, и т.д.)     |
| `cpi_uah`             | `NUMERIC(6,2)` | Стоимость установки (Cost Per Install) в грн |

> **`cpi_uah` (Cost Per Install)** - это сумма, которую компания платит рекламной платформе (например, Facebook, Google Ads) за то, что пользователь установил приложение по рекламе.
> * Для `Organic` установок значение обычно `0`
> * Используется для оценки эффективности каналов и расчёта окупаемости (ROI)

---

**2. `product_views_log`** — просмотры товаров

| Поле          | Тип SQL       | Описание                         |
| ------------- | ------------- | -------------------------------- |
| `device_code` | `VARCHAR(20)` | Устройство                       |
| `view_date`   | `DATE`        | Дата просмотра                   |
| `platform`    | `VARCHAR(10)` | Платформа                        |
| `view_count`  | `INTEGER`     | Количество просмотренных товаров |

---

**3. `devices_users_map`** — соответствие `device_code` и `user_uuid`

| Поле          | Тип SQL       | Описание                                |
| ------------- | ------------- | --------------------------------------- |
| `device_code` | `VARCHAR(20)` | Устройство                              |
| `user_uuid`   | `VARCHAR(20)` | Пользователь (присваивается при логине) |

> 🔎 Пользователь может не авторизоваться — тогда `user_uuid` будет отсутствовать.

---

**4. `orders_log`** — покупки

| Поле         | Тип SQL         | Описание                                   |
| ------------ | --------------- | ------------------------------------------ |
| `user_uuid`  | `VARCHAR(20)`   | Уникальный ID авторизованного пользователя |
| `order_time` | `DATE`          | Дата заказа                                |
| `total_uah`  | `NUMERIC(10,2)` | Сумма покупки в гривнах                    |

---

| Связь                                                      | Описание                          |
| ---------------------------------------------------------- | --------------------------------- |
| `app_sessions.device_code = devices_users_map.device_code` | Склейка установки с пользователем |
| `devices_users_map.user_uuid = orders_log.user_uuid`       | Кто сделал заказ                  |
| `product_views_log.device_code = app_sessions.device_code` | Кто смотрел товары                |

---

### Путь пользователя

```text
УСТАНОВИЛ → СМОТРЕЛ → АВТОРИЗОВАЛСЯ → КУПИЛ
(app_sessions) → (product_views_log) → (devices_users_map) → (orders_log)
```

## SQL - запросы на создание таблиц

```sql
DROP TABLE IF EXISTS orders_log;
DROP TABLE IF EXISTS devices_users_map;
DROP TABLE IF EXISTS product_views_log;
DROP TABLE IF EXISTS app_sessions;

CREATE TABLE app_sessions (
    session_id VARCHAR(10) PRIMARY KEY,
    device_code VARCHAR(20) UNIQUE,
    first_seen DATE,
    os_type VARCHAR(10),
    acquisition_channel VARCHAR(50),
    cpi_uah NUMERIC(6, 2)
);

CREATE TABLE product_views_log (
    device_code VARCHAR(20),
    view_date DATE,
    platform VARCHAR(10),
    view_count INTEGER,
    FOREIGN KEY (device_code) REFERENCES app_sessions(device_code)
);

CREATE TABLE devices_users_map (
    device_code VARCHAR(20),
    user_uuid VARCHAR(20) UNIQUE,
    FOREIGN KEY (device_code) REFERENCES app_sessions(device_code)
);

CREATE TABLE orders_log (
    user_uuid VARCHAR(20),
    order_time DATE,
    total_uah NUMERIC(10, 2),
    FOREIGN KEY (user_uuid) REFERENCES devices_users_map(user_uuid)
);
```

## Подключение к БД

## SELECT, FROM, WHERE — основа SQL-запроса

Каждый SQL-запрос начинается с **четкой структуры**:

```sql
SELECT [что выбрать]
FROM [откуда взять]
WHERE [какие строки отфильтровать]
```

### 🔹 SELECT

* Указывает, **какие столбцы** мы хотим получить
* Можно указать `*`, чтобы взять все столбцы (на практике — только для отладки)
* Поддерживает **выражения** (арифметика, функции, переименование через `AS`)

> ❗ **Порядок строк не гарантируется!**
> Если вам важен порядок — используйте `ORDER BY`

### 🔹 FROM

* Обязательно указывает, **из какой таблицы** брать данные
* Может быть не только одна таблица (будет позже, с `JOIN`)

### 🔹 WHERE

* Фильтрация строк до всех остальных операций (до `GROUP BY`, `HAVING`)
* Работает только со строками, которые **есть** в таблице

> ❗ **WHERE не может использовать агрегатные функции** (например, `AVG()`)

---

⚠️ **Пример небезопасного кода (SQL-инъекция):**

Представим, что ты вставляешь значение напрямую в строку запроса (так делать нельзя!):

```python
user_input = "2024-03-01' OR '1'='1"
query = f"""
SELECT * FROM app_sessions
WHERE first_seen >= '{user_input}'
"""

print(query)  # Вывод запроса

df = pd.read_sql(query, con=engine)  # ⚠️ Опасно!
```

👉 Что получится:

```sql
SELECT * FROM app_sessions
WHERE first_seen >= '2024-03-01' OR '1'='1'
```

🔴 **Что делает эта "инъекция"?**
Условие `OR '1'='1'` всегда истинно → фильтр `first_seen >= ...` перестаёт работать.
**Результат: возвращаются все строки** — потенциальная утечка данных.

## Логические операторы AND, OR, IN, BETWEEN, LIKE

#### `AND`, `OR`

* Позволяют соединять условия
* `AND` — обе части должны быть True
* `OR` — хотя бы одна часть True

> 📌 **Используйте скобки!** Логика без скобок может вас подвести

#### `IN (...)`

* Упрощает множественные сравнения
* Альтернатива множеству `OR`

#### `BETWEEN a AND b`

* Диапазон значений (включительно)
* Удобно для фильтрации по датам, числам

#### `LIKE`

* Шаблон поиска (например, `"Goo%"`)
* `%` — любое количество любых символов
* `_` — ровно один любой символ

##### `LIKE` — **чувствителен к регистру** в **PostgreSQL**. Относительно других СУБД - нужно проверять.

```sql
SELECT * FROM users WHERE name LIKE 'Alex';
```

**Найдет только 'Alex'**, но **не** 'alex', 'ALEX' и т.п.

---

Если хочешь **регистронезависимый поиск**, используй:

1. `ILIKE` (PostgreSQL only)

```sql
SELECT * FROM users WHERE name ILIKE 'alex%';
```

Найдет `Alex`, `alex`, `ALEX`, `AlEx` и т.д.

2. `LOWER()` или `UPPER()`:

```sql
SELECT * FROM users WHERE LOWER(name) LIKE LOWER('alex%');
```

##### **%** = любой набор символов (в том числе пустой)

| Условие        | Найдёт строки, в которых...                                              |
| -------------- | ------------------------------------------------------------------------ |
| `LIKE 'Alex%'` | ...начинается с `'Alex'` (например: `Alex`, `Alexander`, `Alex123`)      |
| `LIKE '%son'`  | ...заканчивается на `'son'` (например: `Jackson`, `Emerson`)             |
| `LIKE '%lex%'` | ...содержится `'lex'` в любом месте (например: `Alex`, `Flex`, `Reflex`) |
| `LIKE '%'`     | ...все строки (потому что «любое количество любых символов»)             |

```sql
SELECT * FROM products
WHERE name LIKE '%phone%';
```

Найдёт: `smartphone`, `Phone Case`, `Headphones`, `my_phone_123`.

---

### Замена на `ILIKE` (для нечувствительного к регистру поиска):

```sql
SELECT * FROM products
WHERE name ILIKE '%phone%';
```

##### Символ `_` (один любой символ):

| Условие       | Найдёт                             |
| ------------- | ---------------------------------- |
| `LIKE '_ex'`  | `Lex`, `Rex`, но **не** `Alex`     |
| `LIKE 'A__x'` | `Alex`, `Abbx`, но не `Ax`, `Alxx` |

---

## ORDER BY, LIMIT


### `ORDER BY`

* Сортирует результат по указанному столбцу
* По умолчанию: **по возрастанию (`ASC`)**
* Указать `DESC` — по убыванию

> ❗ Без `ORDER BY` никакой порядок НЕ ГАРАНТИРУЕТСЯ, даже если таблица физически так хранится

### `LIMIT`

* Ограничивает количество строк
* Часто используется с `ORDER BY`, чтобы получить "топ-N"

## IS NULL — работа с пропущенными значениями

* В SQL значение `NULL` — это **"ничего"**, "неизвестно"
* Сравнивать `= NULL` нельзя! Используется `IS NULL` и `IS NOT NULL`

> ❗ `NULL` не участвует в арифметике и логике — результат всегда `NULL`

| Операция        | `NULL` даёт...     | Вместо этого             |
| --------------- | ------------------ | ------------------------ |
| `= NULL`        | `NULL` (не `TRUE`) | `IS NULL`                |
| `NULL + 1`      | `NULL`             | `COALESCE(col, 0) + 1`   |
| `NULL AND TRUE` | `NULL`             | Будь осторожен в `WHERE` |

---

## DISTINCT — убрать дубликаты

* Убирает повторяющиеся строки по указанным столбцам
* Можно использовать в подзапросах или для подсчета уникальных значений

> ❗ `DISTINCT` всегда влияет на **всю комбинацию столбцов**, а не по отдельности

Оператор `DISTINCT` в SQL **удаляет дубликаты** — но он работает на **всю строку результата**, а не на каждый столбец по отдельности.

**Пример:**

| order\_id | user\_id | order\_time |
| --------- | -------- | ----------- |
| 1         | 101      | 2024-06-01  |
| 2         | 102      | 2024-06-01  |
| 3         | 101      | 2024-06-02  |
| 4         | 101      | 2024-06-01  |

---

```sql
SELECT DISTINCT user_id, order_time FROM orders_log;
```

Это вернёт **уникальные сочетания** `(user_id, order_time)`.

---

| user\_id | order\_time |
| -------- | ----------- |
| 101      | 2024-06-01  |
| 102      | 2024-06-01  |
| 101      | 2024-06-02  |

